In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [2]:
data_dir = "../data/SimpleQuestions_v2_modified/"
train_path = os.path.join(data_dir, "train.txt")
val_path = os.path.join(data_dir, "val.txt")
test_path = os.path.join(data_dir, "test.txt")

In [3]:
train_df = pd.read_table(train_path, header=None, names=["lineid", "entity_mid", "relation", "object", "question"])
val_df = pd.read_table(val_path, header=None, names=["lineid", "entity_mid", "relation", "object", "question"])
test_df = pd.read_table(test_path, header=None, names=["lineid", "entity_mid", "relation", "object", "question"])

In [4]:
train_df.head()

,lineid,entity_mid,relation,object,question
0,train-1,fb:m.04whkz5,fb:book.written_work.subjects,fb:m.01cj3p,what is the book e about
1,train-2,fb:m.0tp2p24,fb:music.release_track.release,fb:m.0sjc7c1,to what release does the release track cardiac...
2,train-3,fb:m.04j0t75,fb:film.film.country,fb:m.07ssc,what country was the film the debt from
3,train-4,fb:m.0ftqr,fb:music.producer.tracks_produced,fb:m.0p600l,what songs have nobuo uematsu produced?
4,train-5,fb:m.036p007,fb:music.release.producers,fb:m.0677ng,Who produced eve-olution?


In [ ]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_df['question'])
X_train_counts.shape

In [ ]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

In [5]:
# clf = LogisticRegression().fit(X_train_tfidf, train_df['relation'])
# pipeline...
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression()),
])

In [ ]:
text_clf.fit(train_df['question'], train_df['relation'])

In [ ]:
predicted = text_clf.predict(test_df['question'])

In [ ]:
accuracy = 100.0 * np.mean(predicted == test_df['relation'])